<a href="https://colab.research.google.com/github/osgeokr/kari-sdm/blob/main/005_environmental_variables_trees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import ee

# Earth Engine 인증
ee.Authenticate()

# Earth Engine 초기화
ee.Initialize(project='ee-foss4g')

In [10]:
!pip install -q -U rasterio

In [11]:
import geemap
import geemap.colormaps as cm
import os, requests
import pandas as pd, geopandas as gpd
import matplotlib.pyplot as plt
import rasterio
import numpy as np

from google.colab import files
from google.colab import drive

## Trees

### Canopy Height (수관 높이)

Meta와 세계자원연구소(WRI: World Resources Institute)는 1미터 해상도의 전 세계 수관 높이 지도를 발표하여, 전 지구적으로 단일 나무를 감지할 수 있는 기능을 제공한다. 오픈 소스 산림 모니터링을 촉진하기 위해, 모든 수관 높이 데이터와 인공지능 모델은 무료로 공개된다.

Tolan, J., Yang, H.I., Nosarzewski, B., Couairon, G., Vo, H.V., Brandt, J., Spore, J., Majumdar, S., Haziza, D., Vamaraju, J., & Moutakanni, T. (2024). Creating highly detailed canopy height maps from RGB imagery with a self-supervised vision transformer and convolutional decoder, trained on aerial lidar data. Published in *Remote Sensing of Environment*, 300, p.113888.

https://meta-forest-monitoring-okw37.projects.earthengine.app/view/canopyheight
https://sustainability.atmeta.com/blog/2024/04/22/using-artificial-intelligence-to-map-the-earths-forests/
https://gee-community-catalog.org/projects/meta_trees/

In [12]:
# 대한민국 범위 좌표 지정(좌하단 : 우상단 좌표)
rectangle_coords = [
    [125.0765578311700068, 33.1124998462386984],
    [131.8727812628719960, 38.4000004985049017],
]

# 직사각형 생성
rectangle = ee.Geometry.Rectangle(rectangle_coords)

# 직사각형을 Feature로 변환
feature = ee.Feature(rectangle)

# FeatureCollection 생성
kr = ee.FeatureCollection([feature])

# NASA SRTM Digital Elevation 30m
elevation = ee.Image("USGS/SRTMGL1_003").select("elevation")

# 수역 마스크 생성
watermask = elevation.gt(0)

In [19]:
# Canopy Height 데이터셋 호출 및 모자이크 처리
canopy_height = ee.ImageCollection('projects/meta-forest-monitoring-okw37/assets/CanopyHeight').mosaic()

# canopyHeight >= 1인 지역만 선택
canopy_height = canopy_height.updateMask(canopy_height.gte(1))

# 수역 마스크 적용
canopy_height = canopy_height.updateMask(watermask).clip(kr.geometry())

# 레이어 추가 호출
eth_canopy_height = ee.Image('users/nlang/ETH_GlobalCanopyHeight_2020_10m_v1')
umd_height = ee.ImageCollection("users/potapovpeter/GEDI_V27").mosaic()

# 시각화 파라미터
vis_params = {
    "min": 0,
    "max": 25,
    "palette": ['440154', '3b528b', '21908d', '5ec962', 'fde725'],
}

# 지도 생성 및 레이어 추가
Map = geemap.Map(layout={"height": "400px", "width": "800px"})

# ETH Global Canopy Height Layer 추가 (10m급)
Map.addLayer(eth_canopy_height, vis_params, 'Canopy Height Lang 2022 98%', False)

# UMD Canopy Height Layer 추가 (30m급)
Map.addLayer(umd_height, vis_params, 'Canopy Height Potapov 2021 95%', False)

# Global Canopy Height Layer 추가 (1m급)
Map.addLayer(canopy_height, vis_params, 'Canopy Height [meters]')

Map.centerObject(kr.geometry(), 6)
Map

Map(center=[35.77459004930315, 128.47466954702105], controls=(WidgetControl(options=['position', 'transparent_…